In [39]:
# 내 구글 드라이브에 연동
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [40]:
# 필요한 패키지와 모듈을 불러옴
import numpy as np
import dlib
import cv2
import time

In [41]:
# eyes landmark 를 정의합니다.
RIGHT_EYE = list(range(36, 42))  # 오른쪽 눈
LEFT_EYE = list(range(42, 48))   # 왼쪽 눈
EYES = list(range(36, 48))       # 양쪽 눈

In [42]:
detector = dlib.get_frontal_face_detector()       # 얼굴을 감지하는 detector 정의
predictor_file = 'gdrive/My Drive/CV/Face Landmark/shape_predictor_68_face_landmarks.dat'  # 이미 학습된 dlib 모델
predictor = dlib.shape_predictor(predictor_file)   # facial landmark 을 찾아주는 객체 생성

In [43]:
file_name = 'eye1.mp4'           # 원본 동영상 파일
#file_name = 'gdrive/My Drive/CV/Drowsiness Detection/drowsiness.mp4'           # 원본 동영상 파일
output_name = 'drowsiness_detection_video.mp4'   # landmark 된 output 동영상 이름

In [44]:
status = 'Awake'   # status 초기화
number_closed = 0  # number_closed 초기화
min_EAR = 0.21     # 눈 뜬 간격 최소화 설정
closed_limit = 7   # 눈을 감은 patience 수
txt = None         # text 문구 초기화
color = None       # text 문구 글자색 초기화
elapsed_time = 0   # 동영상 detection 하는데 걸린 총 경과시간 초기화

In [45]:
# EAR(eye aspect ratio) 계산식으로 눈이 떠져 있는 계수를 계산해 낸다(보통 0.2~0.3 이상이 눈을 뜨고 있다고 판단)
def getEAR(points):   
    A = np.linalg.norm(points[1] - points[5])
    B = np.linalg.norm(points[2] - points[4])
    C = np.linalg.norm(points[0] - points[3])
    return (A + B) / (2.0 * C)

def getFaceDimension(rect):
    return (rect.left(), rect.top(), rect.right() - rect.left(), rect.bottom() - rect.top())

In [46]:
def detectAndDisplay(image):
    global number_closed
    global color
    global txt
    global elapsed_time
    
    start_time = time.time()
    
    frame_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    faces = detector(frame_gray, 1)

    for (i, face) in enumerate(faces):
      x, y, w, h = getFaceDimension(face)
      
    x_p, y_p = x, y  # box 시작 위치 좌표 저장

    cv2.rectangle(image, (x, y), (x+w, y+h), (0, 255, 255), 2)  # 이미지에 box 를 그린다.
    
    rect = dlib.rectangle(x, y, x + w, y + h)  # dlib 에서 사용하는 rect 좌표 형식을 맞춘다.

    points = np.matrix([[p.x, p.y] for p in predictor(frame_gray, rect).parts()])  # 68개 point 좌표를 찾아온다.
    show_parts = points[EYES]     # EYES points 좌표만 찾아온다.
    right_eye_EAR = getEAR(points[RIGHT_EYE])          # 오른쪽 눈의 EAR 계수 계산
    left_eye_EAR = getEAR(points[LEFT_EYE])            # 왼쪽 눈의 EAR 계수 계산
    mean_eye_EAR = (right_eye_EAR + left_eye_EAR) / 2  # 양쪽 눈의 EAR 계수 평균값 계산

    right_eye_center = np.mean(points[RIGHT_EYE], axis = 0).astype("int")  # 오른쪽 눈의 중앙지점 계산
    left_eye_center = np.mean(points[LEFT_EYE], axis = 0).astype("int")    # 왼쪽 눈의 중앙지점 계산

    cv2.putText(image, "{:.2f}".format(right_eye_EAR), (right_eye_center[0,0], right_eye_center[0,1] + 20),
        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)   # 오른쪽 눈의 EAR 계수 Display
    cv2.putText(image, "{:.2f}".format(left_eye_EAR), (left_eye_center[0,0], left_eye_center[0,1] + 20),
        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)   # 왼쪽 눈의 EAR 계수 Display
    
    # 이미지에 눈을 표현해 주기 위해 landmark point 를 그린다.
    for (i, point) in enumerate(show_parts):
        x = point[0,0]
        y = point[0,1]
        cv2.circle(image, (x, y), 1, (0, 255, 255), -1)

    # 눈을 뜨고 있는 경우    
    if mean_eye_EAR > min_EAR:
        color = (0, 255, 0)
        status = 'Awake'
        number_closed = number_closed - 1
        if( number_closed < 0 ):
            number_closed = 0
    else:    # 눈을 감고 있는 경우
        color = (0, 0, 255)
        status = 'Sleep'
        number_closed = number_closed + 1
                  
    txt = status + ', Sleep count : ' + str(number_closed) + ' / ' + str(closed_limit)
    cv2.putText(image, txt, (x_p+3,y_p-7), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

    # video 를 disk 에 output 하기 위해 writer 를 초기화한다.
    global writer
    if writer is None and output_name is not None:
      fourcc = cv2.VideoWriter_fourcc(*"MJPG")
      writer = cv2.VideoWriter(output_name, fourcc, 30, (image.shape[1], image.shape[0]), True)  # 컬러 영상 저장
      
    # disk 에 image 을 write 합니다.
    if writer is not None:
      writer.write(image)

    frame_time = time.time() - start_time   # frame 당 처리시간
    print("Frame time {:.3f} seconds".format(frame_time))
    elapsed_time += frame_time              # 총 경과 시간 누적

In [47]:
cap = cv2.VideoCapture(file_name)     # 동영상을 읽어옴

writer = None
# 원본 동영상이 오픈되는지 확인
if not cap.isOpened:
    print('--(!)Error opening video capture')
    exit(0)

In [48]:
while True:
  # 원본 동영상에서 frame 을 읽는다
  ret, image = cap.read()

  # 원본 동영상에서 더 이상 frame 을 읽지 못했다면 Exit
  if image is None:
     # close the video file pointers
     cap.release()
     # close the writer point
     writer.release()
     print('--(!) No captured frame -- Break!')
     print("elapsed time {:.3f} seconds".format(elapsed_time))
     break

  detectAndDisplay(image)

Frame time 0.393 seconds
Frame time 0.391 seconds
Frame time 0.392 seconds
Frame time 0.394 seconds
Frame time 0.399 seconds
Frame time 0.393 seconds
Frame time 0.389 seconds
Frame time 0.394 seconds
Frame time 0.403 seconds
Frame time 0.396 seconds
Frame time 0.388 seconds
Frame time 0.392 seconds
Frame time 0.395 seconds
Frame time 0.388 seconds
Frame time 0.402 seconds
Frame time 0.392 seconds
Frame time 0.396 seconds
Frame time 0.395 seconds
Frame time 0.394 seconds
Frame time 0.399 seconds
Frame time 0.415 seconds
Frame time 0.394 seconds
Frame time 0.391 seconds
Frame time 0.389 seconds
Frame time 0.416 seconds
Frame time 0.388 seconds
Frame time 0.396 seconds
Frame time 0.392 seconds
Frame time 0.396 seconds
Frame time 0.392 seconds
Frame time 0.393 seconds
Frame time 0.397 seconds
Frame time 0.399 seconds
Frame time 0.388 seconds
Frame time 0.394 seconds
Frame time 0.394 seconds
Frame time 0.393 seconds
Frame time 0.392 seconds
Frame time 0.389 seconds
Frame time 0.408 seconds
